In [1]:
from pyspark import *
from pyspark.sql import SparkSession

In [2]:
sc = SparkContext()
spark = SparkSession(sc)

In [10]:
df = spark.read.csv("NUSW-NB15.csv", inferSchema = True, header = True)

In [11]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- srcip: string (nullable = true)
 |-- sport: string (nullable = true)
 |-- dstip: string (nullable = true)
 |-- dsport: string (nullable = true)
 |-- proto: string (nullable = true)
 |-- state: string (nullable = true)
 |-- dur: double (nullable = true)
 |-- sbytes: integer (nullable = true)
 |-- dbytes: integer (nullable = true)
 |-- sttl: integer (nullable = true)
 |-- dttl: integer (nullable = true)
 |-- sloss: integer (nullable = true)
 |-- dloss: integer (nullable = true)
 |-- service: string (nullable = true)
 |-- Sload: double (nullable = true)
 |-- Dload: double (nullable = true)
 |-- Spkts: integer (nullable = true)
 |-- Dpkts: integer (nullable = true)
 |-- swin: integer (nullable = true)
 |-- dwin: integer (nullable = true)
 |-- stcpb: long (nullable = true)
 |-- dtcpb: long (nullable = true)
 |-- smeansz: integer (nullable = true)
 |-- dmeansz: integer (nullable = true)
 |-- trans_depth: integer (nullable = true)
 |-- res_bdy_len

In [12]:
df.count()

700000

In [17]:
df.groupby("srcip").count().filter("`count` >= 10000").orderBy('count', ascending=False).show()

+------------+-----+
|       srcip|count|
+------------+-----+
|  59.166.0.2|67209|
|  59.166.0.0|67127|
|  59.166.0.5|67091|
|  59.166.0.4|66722|
|  59.166.0.1|66587|
|  59.166.0.3|66145|
|  59.166.0.6|64689|
|  59.166.0.8|64640|
|  59.166.0.9|64187|
|  59.166.0.7|63725|
|175.45.176.1|14325|
+------------+-----+



In [23]:
df.groupby("dstip").count().filter("`count` >= 10000").orderBy('count', ascending=False).show()

+-------------+-----+
|        dstip|count|
+-------------+-----+
|149.171.126.4|67331|
|149.171.126.1|66973|
|149.171.126.0|66925|
|149.171.126.5|66723|
|149.171.126.3|66697|
|149.171.126.2|66674|
|149.171.126.9|65187|
|149.171.126.7|64057|
|149.171.126.6|63951|
|149.171.126.8|63604|
+-------------+-----+



In [28]:
df.select('srcip','Spkts','Dpkts').show()

+------------+--------------+-----+-----+
|       srcip|         dstip|Spkts|Dpkts|
+------------+--------------+-----+-----+
|  59.166.0.0| 149.171.126.9|    4|    4|
|  59.166.0.6| 149.171.126.7|    2|    2|
|  59.166.0.5| 149.171.126.5|    2|    2|
|  59.166.0.3| 149.171.126.0|    2|    2|
|  59.166.0.0| 149.171.126.9|    4|    4|
|  59.166.0.6| 149.171.126.4|    2|    2|
| 10.40.182.3|   10.40.182.3|    1|    0|
|  59.166.0.5| 149.171.126.6|    2|    2|
|  59.166.0.7| 149.171.126.4|    2|    2|
| 10.40.170.2|   10.40.170.2|    1|    0|
| 10.40.170.2|   10.40.170.2|    1|    0|
| 10.40.182.3|   10.40.182.3|    1|    0|
|  59.166.0.1| 149.171.126.5|    2|    2|
|  59.166.0.1| 149.171.126.9|    4|    4|
|  59.166.0.1| 149.171.126.6|    4|    4|
|  59.166.0.2| 149.171.126.4|    2|    2|
|  59.166.0.1| 149.171.126.9|    4|    4|
|  59.166.0.1| 149.171.126.6|    4|    4|
|  59.166.0.4| 149.171.126.5|    2|    2|
|175.45.176.3|149.171.126.18|    2|    0|
+------------+--------------+-----

In [32]:
srcip = df.select('srcip', 'Spkts', 'Dpkts').groupby('srcip').sum()
srcip.show()

+--------------+----------+----------+
|         srcip|sum(Spkts)|sum(Dpkts)|
+--------------+----------+----------+
|149.171.126.16|        18|         6|
|    59.166.0.5|   2940794|   3963509|
|   10.40.170.2|       934|         0|
|    59.166.0.8|   2780578|   3697291|
|    10.40.85.1|      7460|      1626|
|149.171.126.18|     21220|         8|
|    59.166.0.3|   2723587|   3552857|
| 149.171.126.7|      8714|      5820|
|  175.45.176.1|    127322|     66037|
|149.171.126.15|        40|        28|
| 149.171.126.4|     10436|      6708|
|    59.166.0.6|   2839878|   3789330|
|    59.166.0.4|   2822000|   3710255|
|149.171.126.17|         2|         0|
| 149.171.126.2|      9577|      6475|
|149.171.126.13|         4|         4|
|149.171.126.10|        14|        10|
|    59.166.0.1|   2891346|   3883201|
| 149.171.126.3|      8741|      6228|
|    59.166.0.0|   2912738|   3900524|
+--------------+----------+----------+
only showing top 20 rows



In [33]:
srcip.filter("`sum(Spkts)` >= 100000").orderBy('sum(Spkts)', ascending=False).show()

+------------+----------+----------+
|       srcip|sum(Spkts)|sum(Dpkts)|
+------------+----------+----------+
|  59.166.0.2|   2947899|   3968021|
|  59.166.0.5|   2940794|   3963509|
|  59.166.0.0|   2912738|   3900524|
|  59.166.0.9|   2899288|   3955044|
|  59.166.0.1|   2891346|   3883201|
|  59.166.0.7|   2877707|   3926245|
|  59.166.0.6|   2839878|   3789330|
|  59.166.0.4|   2822000|   3710255|
|  59.166.0.8|   2780578|   3697291|
|  59.166.0.3|   2723587|   3552857|
|175.45.176.1|    127322|     66037|
|175.45.176.2|    119870|     97087|
|175.45.176.0|    100482|     73569|
+------------+----------+----------+

